### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2023
quarter = 1

today = date.today()
today

datetime.date(2023, 7, 4)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2023 AND quarter <= 1) 
OR (year = 2023-1 AND quarter >= 1+1)
ORDER BY year DESC, quarter DESC


In [3]:
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp.shape

(214, 5)

In [4]:
sql = '''
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC'''
sql = sql % (year,quarter,year,quarter)
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(229, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct.sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
49,CRC,2023,Q1,8139363,861862,7277501,844.39
70,GRAMMY,2023,Q1,321728,-54729,376457,687.86
177,TFG,2023,Q1,4528836,734740,3794096,516.39
124,PLANB,2023,Q1,731042,131262,599780,456.93
142,SABUY,2023,Q1,1541775,287207,1254568,436.82


In [6]:
sql = """
DELETE FROM yr_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year, quarter)
rp = conlt.execute(sql)
rp.rowcount

211

In [7]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
rcds = df_ins.values.tolist()
len(rcds)

212

In [8]:
for rcd in rcds:
    print(rcd)

['ACE', 2023, 'Q1', 1234293, 1468062, -233769, -15.92, 698]
['ADVANC', 2023, 'Q1', 26457375, 26589096, -131721, -0.5, 6]
['AH', 2023, 'Q1', 1983003, 1015701, 967302, 95.23, 9]
['AIE', 2023, 'Q1', -151353, 432491, -583844, -135.0, 720]
['AIMIRT', 2023, 'Q1', 682646, 524978, 157668, 30.03, 669]
['AIT', 2023, 'Q1', 472255, 571977, -99722, -17.43, 11]
['AJ', 2023, 'Q1', -306699, 467710, -774409, -165.57, 12]
['AMATA', 2023, 'Q1', 2279873, 1770024, 509849, 28.8, 21]
['ANAN', 2023, 'Q1', -74085, -744140, 670055, 90.04, 23]
['AOT', 2023, 'Q1', -6473437, -17151697, 10678260, 62.26, 24]
['AP', 2023, 'Q1', 5625945, 4870212, 755733, 15.52, 25]
['ASIAN', 2023, 'Q1', 780969, 1077825, -296856, -27.54, 36]
['ASK', 2023, 'Q1', 1563900, 1277644, 286256, 22.4, 38]
['ASP', 2023, 'Q1', 460634, 775000, -314366, -40.56, 40]
['ASW', 2023, 'Q1', 1175597, 856966, 318631, 37.18, 728]
['AWC', 2023, 'Q1', 4630796, 2100590, 2530206, 120.45, 699]
['BA', 2023, 'Q1', -215102, -8824265, 8609163, 97.56, 45]
['BAM', 202

In [9]:
sql = """INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)

INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [10]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [11]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [12]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
49,CRC,2023,Q1,8139363,861862,7277501,844.39
177,TFG,2023,Q1,4528836,734740,3794096,516.39
124,PLANB,2023,Q1,731042,131262,599780,456.93
142,SABUY,2023,Q1,1541775,287207,1254568,436.82
106,M,2023,Q1,1493308,313003,1180305,377.09


In [13]:
criteria_2 = df_ins.q_amt_p > 300000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
49,CRC,2023,Q1,8139363,861862,7277501,844.39
177,TFG,2023,Q1,4528836,734740,3794096,516.39
106,M,2023,Q1,1493308,313003,1180305,377.09
30,BH,2023,Q1,5796547,1849587,3946960,213.40
66,GFPT,2023,Q1,1886904,604464,1282440,212.16


In [14]:
criteria_3 = df_ins.Pct > 5.0
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
49,CRC,2023,Q1,8139363,861862,7277501,844.39
70,GRAMMY,2023,Q1,321728,-54729,376457,687.86
177,TFG,2023,Q1,4528836,734740,3794096,516.39
124,PLANB,2023,Q1,731042,131262,599780,456.93
142,SABUY,2023,Q1,1541775,287207,1254568,436.82


In [15]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
49,CRC,2023,Q1,8139363,861862,7277501,844.39
177,TFG,2023,Q1,4528836,734740,3794096,516.39
106,M,2023,Q1,1493308,313003,1180305,377.09
30,BH,2023,Q1,5796547,1849587,3946960,213.40
66,GFPT,2023,Q1,1886904,604464,1282440,212.16
154,SIRI,2023,Q1,5559109,1935771,3623338,187.18
27,BEM,2023,Q1,2849148,1040915,1808233,173.72
47,CPNREIT,2023,Q1,2279637,869788,1409849,162.09
96,KSL,2023,Q1,1526784,639300,887484,138.82
109,MC,2023,Q1,832309,360228,472081,131.05


In [16]:
df_ins.loc[df_ins_criteria,cols].sort_values(['name'],ascending=[True])

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,AH,2023,Q1,1983003,1015701,967302,95.23
4,AIMIRT,2023,Q1,682646,524978,157668,30.03
7,AMATA,2023,Q1,2279873,1770024,509849,28.80
10,AP,2023,Q1,5625945,4870212,755733,15.52
12,ASK,2023,Q1,1563900,1277644,286256,22.40
14,ASW,2023,Q1,1175597,856966,318631,37.18
15,AWC,2023,Q1,4630796,2100590,2530206,120.45
18,BANPU,2023,Q1,35242778,18580896,16661882,89.67
20,BBL,2023,Q1,32316817,26702007,5614810,21.03
22,BCP,2023,Q1,10959827,9696604,1263223,13.03
